## Final Project
### Visual data

Scripts of analysis data. Used SparkSQL.

In [1]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import csv

from datetime import *
from dateutil.parser import parse

sqlContext = SQLContext(sc)
def ParseDate(t):
    try:
        dt = parse(t, fuzzy=True).date()
    except ValueError as e:
        dt = date.today()
    return dt

In [4]:
# read the file from loacal
crimeFile = sc.textFile("file:///home/ubuntu/crime/raw_data/row06-15.csv")

# get the headler from csv
header = crimeFile.first().split(',')

# choose the fileds that will used in the future
fields = [StructField(header[i], StringType(), True) for i in [1, 7, 14]]

# change the type of some fileds
fields[0].dataType = DateType()
fields[2].dataType = IntegerType()

# create the schema from the fields
schema = StructType(fields)

# Isolate the header and drop it off the actual data
crimeHeader = crimeFile.filter(lambda l: "CMPLNT_NUM" in l)
crimeNoHeader = crimeFile.subtract(crimeHeader)

# using Spark's rdd.toDF() method, build the dataframe directly from crimeNoHeader RDD
crime_df = crimeNoHeader.mapPartitions(lambda x: csv.reader(x, delimiter=",")).map(lambda row:(ParseDate(row[1]), row[7], int('0'+row[14]))).filter(lambda k: k[0].year>2015 and k[2]>0).toDF(schema)        


In [6]:
crime_df.printSchema()

root
 |-- CMPLNT_FR_DT: date (nullable = true)
 |-- OFNS_DESC: string (nullable = true)
 |-- ADDR_PCT_CD: integer (nullable = true)



In [7]:
crime_df.registerTempTable("crime")
sqlContext.sql('select ADDR_PCT_CD, count(CMPLNT_FR_DT)/count(distinct CMPLNT_FR_DT) as crime_count from crime where YEAR(CMPLNT_FR_DT)=2016 group by ADDR_PCT_CD order by crime_count desc ').show()    

+-----------+------------------+
|ADDR_PCT_CD|       crime_count|
+-----------+------------------+
|         75|40.693430656934304|
|         40|33.496350364963504|
|         43|32.003649635036496|
|         44|31.266423357664234|
|         14| 27.94160583941606|
|         47|27.686131386861312|
|         67|27.412408759124087|
|         52|26.062043795620436|
|         73|25.598540145985403|
|         46| 25.28102189781022|
|        114|24.080291970802918|
|        113|22.025547445255473|
|         42| 21.98905109489051|
|         18| 21.90875912408759|
|        120|              21.0|
|        109|20.806569343065693|
|         48| 20.76277372262774|
|        115|20.554744525547445|
|        103|20.543795620437955|
|         41|20.284671532846716|
+-----------+------------------+
only showing top 20 rows



In [12]:
sqlContext.sql('select MONTH(CMPLNT_FR_DT), count(CMPLNT_FR_DT)/count(distinct YEAR(CMPLNT_FR_DT)) as crime_count from crime where YEAR(CMPLNT_FR_DT)=2016 group by MONTH(CMPLNT_FR_DT) order by MONTH(CMPLNT_FR_DT) ').show()    

+-------------------+-----------+
|month(CMPLNT_FR_DT)|crime_count|
+-------------------+-----------+
|                  1|    37679.0|
|                  2|    34430.0|
|                  3|    39393.0|
|                  4|    39230.0|
|                  5|    41463.0|
|                  6|    41158.0|
|                  7|    40901.0|
|                  8|    41405.0|
|                  9|    36393.0|
+-------------------+-----------+



In [16]:
# Process the temperature
# Load the historical temperature for the city and filter it for the years 2005 to 2015
fields = [StructField('DATE', DateType(), True), StructField('DAYOFWEEK', IntegerType(), True), StructField('TEMP', FloatType(), True)]
schema = StructType(fields)
temperature_df = sc.textFile('file:///home/mona/Documents/BigData/crime/raw_data/NYNEWYOR.txt').map(lambda line: [float(i) for i in line.split()]).filter(lambda row: row[2]>2015 ).map(lambda row: (date(int(row[2]), int(row[0]), int(row[1])), row[3])).map(lambda row: (row[0], row[0].weekday(), row[1])).toDF(schema)


In [17]:
temperature_df.printSchema()

root
 |-- DATE: date (nullable = true)
 |-- DAYOFWEEK: integer (nullable = true)
 |-- TEMP: float (nullable = true)



In [18]:
joined_df = crime_df.join(temperature_df, crime_df.CMPLNT_FR_DT == temperature_df.DATE, 'inner').drop(temperature_df.DATE)
joined_df.printSchema()

root
 |-- CMPLNT_FR_DT: date (nullable = true)
 |-- OFNS_DESC: string (nullable = true)
 |-- ADDR_PCT_CD: integer (nullable = true)
 |-- DAYOFWEEK: integer (nullable = true)
 |-- TEMP: float (nullable = true)



In [19]:
joined_df.registerTempTable("crime")

In [24]:
# sqlContext.sql("select MONTH(CMPLNT_FR_DT), count(CMPLNT_FR_DT)/count(distinct YEAR(CMPLNT_FR_DT)) as crime_count from crime where YEAR(CMPLNT_FR_DT)=2016 group by MONTH(CMPLNT_FR_DT) order by MONTH(CMPLNT_FR_DT) ").show()    
# calculate temperature vs crimes
sqlContext.sql("select max(TEMP), min(TEMP) from crime").show()
print "temperture (10,20]"
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>10 and TEMP<=20").show()
print "temperture (20,30]"
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>20 and TEMP<=30").show()
print "temperture (30,40]"
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>30 and TEMP<=40").show()
print "temperture (40,50]"
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>40 and TEMP<=50").show()
print "temperture (50,60]"
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>50 and TEMP<=60").show()
print "temperture (60,70]"
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>60 and TEMP<=70").show()
print "temperture (70,80]"
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>70 and TEMP<=80").show()
print "temperture (80,90]"
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>80 and TEMP<=90").show()


+---------+---------+
|max(TEMP)|min(TEMP)|
+---------+---------+
|     90.7|      8.3|
+---------+---------+

temperture (10,20]
+--------------------------------------------------------------------------------------------------+
|(CAST(count(CMPLNT_FR_DT) AS DOUBLE) / CAST(count(DISTINCT CAST(CMPLNT_FR_DT AS DATE)) AS DOUBLE))|
+--------------------------------------------------------------------------------------------------+
|                                                                                            1013.0|
+--------------------------------------------------------------------------------------------------+

temperture (20,30]
+--------------------------------------------------------------------------------------------------+
|(CAST(count(CMPLNT_FR_DT) AS DOUBLE) / CAST(count(DISTINCT CAST(CMPLNT_FR_DT AS DATE)) AS DOUBLE))|
+--------------------------------------------------------------------------------------------------+
|                                         

In [25]:
print "temperture (80,90]"
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>80 and TEMP<=90").show()


temperture (80,90]
+--------------------------------------------------------------------------------------------------+
|(CAST(count(CMPLNT_FR_DT) AS DOUBLE) / CAST(count(DISTINCT CAST(CMPLNT_FR_DT AS DATE)) AS DOUBLE))|
+--------------------------------------------------------------------------------------------------+
|                                                                                          1333.125|
+--------------------------------------------------------------------------------------------------+



In [10]:
#sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>80 and TEMP<=90 and OFNS_DESC='' ").show()
sqlContext.sql("select OFNS_DESC, count(*) as NUM from crime group by OFNS_DESC order by NUM desc").show(50, False)


+------------------------------------+-----+
|OFNS_DESC                           |NUM  |
+------------------------------------+-----+
|PETIT LARCENY                       |59455|
|HARRASSMENT 2                       |48967|
|ASSAULT 3 & RELATED OFFENSES        |39682|
|CRIMINAL MISCHIEF & RELATED OF      |36737|
|GRAND LARCENY                       |30563|
|DANGEROUS DRUGS                     |17154|
|OFF. AGNST PUB ORD SENSBLTY &       |16681|
|FELONY ASSAULT                      |15711|
|ROBBERY                             |11434|
|MISCELLANEOUS PENAL LAW             |10350|
|BURGLARY                            |9488 |
|DANGEROUS WEAPONS                   |8034 |
|OFFENSES AGAINST PUBLIC ADMINI      |6801 |
|VEHICLE AND TRAFFIC LAWS            |4931 |
|GRAND LARCENY OF MOTOR VEHICLE      |4692 |
|INTOXICATED & IMPAIRED DRIVING      |4410 |
|FORGERY                             |4365 |
|SEX CRIMES                          |4105 |
|CRIMINAL TRESPASS                   |2990 |
|THEFT-FRA

In [14]:
# calculate temperature vs crimes
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>10 and TEMP<=20 and `OFNS_DESC`='ROBBERY' ").show()
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>20 and TEMP<=30 and `OFNS_DESC`='ROBBERY' ").show()
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>30 and TEMP<=40 and `OFNS_DESC`='ROBBERY' ").show()
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>40 and TEMP<=50 and `OFNS_DESC`='ROBBERY' ").show()
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>50 and TEMP<=60 and `OFNS_DESC`='ROBBERY' ").show()
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>60 and TEMP<=70 and `OFNS_DESC`='ROBBERY' ").show()
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>70 and TEMP<=80 and `OFNS_DESC`='ROBBERY' ").show()
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>80 and TEMP<=90 and `OFNS_DESC`='ROBBERY' ").show()

+--------------------------------------------------------------------------------------------------+
|(CAST(count(CMPLNT_FR_DT) AS DOUBLE) / CAST(count(DISTINCT CAST(CMPLNT_FR_DT AS DATE)) AS DOUBLE))|
+--------------------------------------------------------------------------------------------------+
|                                                                                              32.0|
+--------------------------------------------------------------------------------------------------+

+--------------------------------------------------------------------------------------------------+
|(CAST(count(CMPLNT_FR_DT) AS DOUBLE) / CAST(count(DISTINCT CAST(CMPLNT_FR_DT AS DATE)) AS DOUBLE))|
+--------------------------------------------------------------------------------------------------+
|                                                                                              37.4|
+-----------------------------------------------------------------------------------------

A way to reduce query... but
```
SELECT "periods"."start_date", 
       "periods"."end_date", 
       SUM(CASE WHEN "balance_transactions"."created" BETWEEN "periods"."start_date" AND "periods"."end_date" THEN "balance_transactions"."fee" ELSE 0.00 END) AS period_sum
  FROM "balance_transactions" 
  JOIN charges ON balance_transactions.source = charges.balance_id 
  JOIN ( SELECT '2013-12-20'::date as start_date, '2014-01-19'::date as end_date UNION ALL
         SELECT '2013-12-21'::date as start_date, '2014-01-20'::date as end_date UNION ALL
         SELECT '2013-12-22'::date as start_date, '2014-01-21'::date as end_date UNION ALL
         SELECT '2013-12-23'::date as start_date, '2014-01-22'::date as end_date UNION ALL
         SELECT '2013-12-24'::date as start_date, '2014-01-23'::date as end_date
         ) as periods
    ON "balance_transactions"."created" BETWEEN "periods"."start_date" AND "periods"."end_date"
 WHERE "balance_transactions"."account_id" = 6 
   AND "balance_transactions"."type" = 'charge' 
   AND "charges"."refunded" = false 
   AND "charges"."invoice" IS NOT NULL
 GROUP BY "periods"."start_date", "periods"."end_date"
```

In [21]:
# calculate daily crime of robbery of weekdays 0-Sun, 1-Mon, 6-Sat
sqlContext.sql("select DAYOFWEEK, count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) as crime_count from crime where `OFNS_DESC`='ROBBERY' group by DAYOFWEEK ").show()    

+---------+-----------------+
|DAYOFWEEK|      crime_count|
+---------+-----------------+
|        1|39.02564102564103|
|        6|45.17948717948718|
|        3| 39.8974358974359|
|        5|42.02564102564103|
|        4|           42.975|
|        2|40.84615384615385|
|        0|42.12820512820513|
+---------+-----------------+



In [22]:
# calculate daily crimes of weekdays 0-Sun, 1-Mon, 6-Sat
sqlContext.sql("select DAYOFWEEK, count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) as crime_count from crime group by DAYOFWEEK ").show()    

+---------+------------------+
|DAYOFWEEK|       crime_count|
+---------+------------------+
|        1|1268.7179487179487|
|        6|1194.7692307692307|
|        3|1307.4102564102564|
|        5|1291.6153846153845|
|        4|          1382.125|
|        2| 1325.948717948718|
|        0| 1220.948717948718|
+---------+------------------+



In [24]:
sqlContext.sql("select DAYOFWEEK, count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) as crime_count from crime where `OFNS_DESC`='PETIT LARCENY' group by DAYOFWEEK ").show()    

+---------+------------------+
|DAYOFWEEK|       crime_count|
+---------+------------------+
|        1|223.82051282051282|
|        6|184.69230769230768|
|        3|226.46153846153845|
|        5|207.66666666666666|
|        4|           227.625|
|        2|228.25641025641025|
|        0|220.12820512820514|
+---------+------------------+



In [23]:
# calculate daily crime of frauds of weekdays 0-Sun, 1-Mon, 6-Sat
sqlContext.sql("select DAYOFWEEK, count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) as crime_count from crime where `OFNS_DESC`='FRAUDS' group by DAYOFWEEK ").show()    

+---------+-----------------+
|DAYOFWEEK|      crime_count|
+---------+-----------------+
|        1|8.743589743589743|
|        6|5.256410256410256|
|        3| 9.41025641025641|
|        5|7.868421052631579|
|        4|             9.35|
|        2|9.384615384615385|
|        0|9.564102564102564|
+---------+-----------------+



In [ ]:
# sqlContext.sql("select MONTH(CMPLNT_FR_DT), count(CMPLNT_FR_DT)/count(distinct YEAR(CMPLNT_FR_DT)) as crime_count from crime where YEAR(CMPLNT_FR_DT)=2016 group by MONTH(CMPLNT_FR_DT) order by MONTH(CMPLNT_FR_DT) ").show()    
# calculate temperature vs crimes
sqlContext.sql("select max(TEMP), min(TEMP) from crime").show()
print "temperture (10,20]"
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>10 and TEMP<=20 group by WEEKDAY").show()
print "temperture (20,30]"
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>20 and TEMP<=30").show()
print "temperture (30,40]"
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>30 and TEMP<=40").show()
print "temperture (40,50]"
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>40 and TEMP<=50").show()
print "temperture (50,60]"
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>50 and TEMP<=60").show()
print "temperture (60,70]"
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>60 and TEMP<=70").show()
print "temperture (70,80]"
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>70 and TEMP<=80").show()
print "temperture (80,90]"
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>80 and TEMP<=90").show()


In [28]:
sqlContext.sql("select DAYOFWEEK, count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>10 and TEMP<=20 group by DAYOFWEEK").show()
sqlContext.sql("select DAYOFWEEK, count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>20 and TEMP<=30 group by DAYOFWEEK").show()
sqlContext.sql("select DAYOFWEEK, count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>30 and TEMP<=40 group by DAYOFWEEK").show()
sqlContext.sql("select DAYOFWEEK, count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>40 and TEMP<=50 group by DAYOFWEEK").show()
sqlContext.sql("select DAYOFWEEK, count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>50 and TEMP<=60 group by DAYOFWEEK").show()
sqlContext.sql("select DAYOFWEEK, count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>60 and TEMP<=70 group by DAYOFWEEK").show()
sqlContext.sql("select DAYOFWEEK, count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>70 and TEMP<=80 group by DAYOFWEEK").show()
sqlContext.sql("select DAYOFWEEK, count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>80 and TEMP<=90 group by DAYOFWEEK").show()


+---------+--------------------------------------------------------------------------------------------------+
|DAYOFWEEK|(CAST(count(CMPLNT_FR_DT) AS DOUBLE) / CAST(count(DISTINCT CAST(CMPLNT_FR_DT AS DATE)) AS DOUBLE))|
+---------+--------------------------------------------------------------------------------------------------+
|        1|                                                                                            1040.0|
|        0|                                                                                             986.0|
+---------+--------------------------------------------------------------------------------------------------+

+---------+--------------------------------------------------------------------------------------------------+
|DAYOFWEEK|(CAST(count(CMPLNT_FR_DT) AS DOUBLE) / CAST(count(DISTINCT CAST(CMPLNT_FR_DT AS DATE)) AS DOUBLE))|
+---------+--------------------------------------------------------------------------------------------------+


In [29]:
# calculate temperature vs crimes
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>10 and TEMP<=20 and `OFNS_DESC`='FRAUDS'  ").show()
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>20 and TEMP<=30 and `OFNS_DESC`='FRAUDS' ").show()
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>30 and TEMP<=40 and `OFNS_DESC`='FRAUDS' ").show()
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>40 and TEMP<=50 and `OFNS_DESC`='FRAUDS' ").show()
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>50 and TEMP<=60 and `OFNS_DESC`='FRAUDS' ").show()
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>60 and TEMP<=70 and `OFNS_DESC`='FRAUDS' ").show()
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>70 and TEMP<=80 and `OFNS_DESC`='FRAUDS' ").show()
sqlContext.sql("select count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>80 and TEMP<=90 and `OFNS_DESC`='FRAUDS' ").show()

+--------------------------------------------------------------------------------------------------+
|(CAST(count(CMPLNT_FR_DT) AS DOUBLE) / CAST(count(DISTINCT CAST(CMPLNT_FR_DT AS DATE)) AS DOUBLE))|
+--------------------------------------------------------------------------------------------------+
|                                                                                               7.0|
+--------------------------------------------------------------------------------------------------+

+--------------------------------------------------------------------------------------------------+
|(CAST(count(CMPLNT_FR_DT) AS DOUBLE) / CAST(count(DISTINCT CAST(CMPLNT_FR_DT AS DATE)) AS DOUBLE))|
+--------------------------------------------------------------------------------------------------+
|                                                                                               7.0|
+-----------------------------------------------------------------------------------------

In [30]:
sqlContext.sql("select DAYOFWEEK, count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>40 and TEMP<=50 and `OFNS_DESC`='FRAUDS' group by DAYOFWEEK").show()
sqlContext.sql("select DAYOFWEEK, count(CMPLNT_FR_DT)/count(distinct DATE(CMPLNT_FR_DT)) from crime where TEMP>40 and TEMP<=50 and `OFNS_DESC`='ROBBERY' group by DAYOFWEEK").show()


+---------+--------------------------------------------------------------------------------------------------+
|DAYOFWEEK|(CAST(count(CMPLNT_FR_DT) AS DOUBLE) / CAST(count(DISTINCT CAST(CMPLNT_FR_DT AS DATE)) AS DOUBLE))|
+---------+--------------------------------------------------------------------------------------------------+
|        1|                                                                                11.571428571428571|
|        6|                                                                                               6.6|
|        3|                                                                                              12.0|
|        5|                                                                                               9.0|
|        4|                                                                                12.666666666666666|
|        2|                                                                                              12.4|
|